In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
directory = 'Dillards_POS/'
skst_file_path = directory + 'skstinfo.csv'
trans_file_path = directory + 'trans.csv'

#sku_file_path = directory + 'skuinfo.csv'
#sku_retail_file_path = directory + 'sku_retail.csv'
clean_sku_file_path = directory + 'clean_sku.csv'

In [3]:
def get_basket(trans_file_path, skst_file_path):
    ''' function that prepares basket from the transaction table'''
    
    # Transaction table
    trans = pd.read_csv(trans_file_path)
    trans.columns = ['sku', 'storeid', 'register', 'trannum', 'interID', 'saledate', 'stype', 'quantity',
                 'orgprice', 'amt', 'seq', 'mic', 'unkown']
    # Drop all returns
    trans = trans[trans['stype'] == 'P']
    
    # Drop columns irrelevant to basket
    trans.drop(columns=['interID', 'stype', 'mic', 'unkown'], inplace=True)
    
    # skst table
    skst = pd.read_csv(skst_file_path, header = None)
    skst.columns = ['sku', 'storeid', 'cost', 'retail', 'unknown']
    
    # Drop columns irrelevant to basket
    skst.drop(columns = ['unknown'], inplace=True)
    
    # Get non zero mean retail for each sku
    mean_retail = skst.groupby('sku')['retail'].unique().reset_index()
    mean_retail['retail'] = mean_retail['retail'].apply(
        lambda x: sum(x) / len(x) if len(x) > 0 else 0 if len([i for i in x if i != 0]) > 0 else 0
    )
    mean_retail = mean_retail.set_index('sku')
    mean_retail_series = mean_retail['retail']
    
    # Merge transaction and skst
    trans = pd.merge(trans, skst, on=['sku','storeid'], how='left')
    
    # Fill NaN values in 'retail' with the mean retail
    trans['retail'] = trans['retail'].fillna(trans['sku'].map(mean_retail_series))
    # Fill NaN values in 'retail' with the corresponding 'orgprice' values
    trans['retail'] = trans['retail'].fillna(trans['orgprice'])
    
    # Fill NaN values in 'orgprice' with the mean retail
    trans['orgprice'] = trans['orgprice'].fillna(trans['sku'].map(mean_retail_series))
    
    
    # Feature engineering
    
    trans['saledate'] = pd.to_datetime(trans['saledate'])
    # day of week as a number (Monday=0, Sunday=6)
    trans['day_of_week'] = trans['saledate'].dt.dayofweek
    # month as a number
    trans['month'] = trans['saledate'].dt.month
    # Define weekend as Friday (4) to Sunday (6)
    trans['weekend'] = trans['day_of_week'].apply(lambda x: 1 if x >= 4 else 0)
    
    # percent discount calculated with (retail - amt) / retail
    trans['percent_discount'] = np.where(
        (trans['retail'] > trans['amt']) & ~np.isnan(trans['retail']) & ~np.isnan(trans['amt']),
        (trans['retail'] - trans['amt']) / trans['retail'],
        0
    )
    # final sale defined as > 50% descount
    trans['final_sale'] = np.where(trans['percent_discount']>0.5, 1, 0)
    return trans

In [4]:
basket = get_basket(trans_file_path, skst_file_path)
basket.head()

,sku,storeid,register,trannum,saledate,quantity,orgprice,amt,seq,cost,retail,day_of_week,month,weekend,percent_discount,final_sale
0,3,202,290,1100,2005-01-18,1,440.0,30.0,3500000,123.36,440.0,1,1,0,0.931818,1
1,3,303,500,2100,2004-08-18,1,440.0,12.0,4600000,NaN,440.0,2,8,0,0.972727,1
2,3,709,360,500,2005-08-14,1,440.0,30.0,6500000,NaN,440.0,6,8,1,0.931818,1
3,3,802,660,400,2005-08-09,1,440.0,30.0,4700000,123.36,440.0,1,8,0,0.931818,1
4,3,1202,400,2100,2004-11-11,1,440.0,30.0,8600000,123.36,440.0,3,11,0,0.931818,1


In [5]:
basket.shape

(111649094, 16)

In [6]:
def join_dataframes(input_df, csv_file_path, columns, join_key = 'sku', how='inner'):
    """
    Joins an input DataFrame with a DataFrame loaded from a CSV file.

    """

    columns = columns  + [join_key]
    # Load the second DataFrame from the CSV file
    csv_df = pd.read_csv(csv_file_path)

    # Perform the join
    joined_df = pd.merge(input_df, csv_df[columns], on=join_key, how=how)

    return joined_df

In [7]:
basket = basket.head(10000)

In [8]:
columns = ['brand', 'classid']
joined_df = join_dataframes(basket, clean_sku_file_path, columns)
joined_df.head()

,sku,storeid,register,trannum,saledate,quantity,orgprice,amt,seq,cost,retail,day_of_week,month,weekend,percent_discount,final_sale,brand,classid
0,3,202,290,1100,2005-01-18,1,440.0,30.0,3500000,123.36,440.0,1,1,0,0.931818,1,TURNBURY,113
1,3,303,500,2100,2004-08-18,1,440.0,12.0,4600000,NaN,440.0,2,8,0,0.972727,1,TURNBURY,113
2,3,709,360,500,2005-08-14,1,440.0,30.0,6500000,NaN,440.0,6,8,1,0.931818,1,TURNBURY,113
3,3,802,660,400,2005-08-09,1,440.0,30.0,4700000,123.36,440.0,1,8,0,0.931818,1,TURNBURY,113
4,3,1202,400,2100,2004-11-11,1,440.0,30.0,8600000,123.36,440.0,3,11,0,0.931818,1,TURNBURY,113
